<a href="https://colab.research.google.com/github/gabrielfernandorey/EDVAI/blob/main/TP%20Final/Edvai_TP_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRABAJO PRACTICO FINAL 
##### EDVAI

## Gabriel Rey 
##### Mayo 2023


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d90b87b8df3e191a21da37214bce95b32efe279d1e43d56b6d43a637cc03c4e3
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

In [7]:
from pyspark.sql.types import StringType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator

from datetime import datetime
import numpy as np
import pandas as pd

### Importamos datos

In [6]:
!wget https://data-engineer-edvai.s3.amazonaws.com/2021-informe-ministerio.csv
!wget https://data-engineer-edvai.s3.amazonaws.com/202206-informe-ministerio.csv
!wget https://data-engineer-edvai.s3.amazonaws.com/aeropuertos_detalle.csv

--2023-05-17 00:00:45--  https://data-engineer-edvai.s3.amazonaws.com/2021-informe-ministerio.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 3.5.1.213, 54.231.160.121, 54.231.161.225, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|3.5.1.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32322556 (31M) [text/csv]
Saving to: ‘2021-informe-ministerio.csv’

2021-informe-minist 100%[===================>]  30.82M  9.29MB/s    in 3.3s    

2023-05-17 00:00:49 (9.29 MB/s) - ‘2021-informe-ministerio.csv’ saved [32322556/32322556]

--2023-05-17 00:00:49--  https://data-engineer-edvai.s3.amazonaws.com/202206-informe-ministerio.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 3.5.1.213, 54.231.160.121, 54.231.161.225, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|3.5.1.213|:443... connecte

In [13]:
df_2021 = pd.read_csv("/content/2021-informe-ministerio.csv", sep=';')

In [14]:
df_2022 = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')

<ipython-input-14-bcad1b5984a1>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')


In [15]:
df_aeropuertos = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')

<ipython-input-15-bc30c10f0059>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aeropuertos = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')


### EDA

In [16]:
df_2021

,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato
0,01/01/2021,00:02,Vuelo Privado con Matrícula Nacional,Domestico,Despegue,PAR,ROS,0,PA-PA-28-181,0,DEFINITIVO
1,01/01/2021,00:24,Regular,Domestico,Aterrizaje,EZE,GRA,AEROLINEAS ARGENTINAS SA,BO-B737-8MB,70,DEFINITIVO
2,01/01/2021,00:26,Regular,Domestico,Aterrizaje,EZE,ECA,AEROLINEAS ARGENTINAS SA,BO-737-800,70,DEFINITIVO
3,01/01/2021,00:29,Regular,Domestico,Aterrizaje,EZE,SAL,AEROLINEAS ARGENTINAS SA,BO-B-737-76N,12,DEFINITIVO
4,01/01/2021,00:37,Regular,Domestico,Aterrizaje,EZE,TUC,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,26,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...
328131,31/12/2021,23:44,Regular,Domestico,Aterrizaje,JUJ,AER,FB LÍNEAS AÉREAS - FLYBONDI,BO-737-8Q8,96,DEFINITIVO
328132,31/12/2021,23:45,Regular,Domestico,Despegue,AER,CBA,FB LÍNEAS AÉREAS - FLYBONDI,BO-737-8Q8,88,DEFINITIVO
328133,31/12/2021,23:48,Regular,Domestico,Despegue,ROS,AER,AEROLINEAS ARGENTINAS SA,BO-737-800,10,DEFINITIVO
328134,31/12/2021,23:55,Regular,Internacional,Aterrizaje,EZE,SCEL,SKY AIRLINE S.A.,0,137,DEFINITIVO


In [19]:
cols = df_2021.columns
cols

Index(['Fecha', 'Hora UTC', 'Clase de Vuelo (todos los vuelos)',
       'Clasificación Vuelo', 'Tipo de Movimiento', 'Aeropuerto',
       'Origen / Destino', 'Aerolinea Nombre', 'Aeronave', 'Pasajeros',
       'Calidad dato'],
      dtype='object')

In [33]:
# Validamos si hay valores faltantes
isna = []
for x in cols:
  isna.append( df_2021[df_2021[x].isna()] )


In [34]:
isna

[Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Ori

In [37]:
# Validamos si hay valores nulos
nulos = []
for x in cols:
  nulos.append( df_2021[df_2021[x].isnull()] )

In [38]:
nulos

[Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Ori

In [39]:
df_2022

,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato
0,01/01/2022,00:01,Regular,Doméstico,Aterrizaje,AER,ECA,AEROLINEAS ARGENTINAS SA,BO-737-8SH,69,DEFINITIVO
1,01/01/2022,00:05,Regular,Doméstico,Aterrizaje,AER,SAL,AEROLINEAS ARGENTINAS SA,BO-B737-8,65,DEFINITIVO
2,01/01/2022,00:05,Regular,Doméstico,Despegue,IGU,AER,JETSMART AIRLINES S.A.,AIB-A320-232,41,DEFINITIVO
3,01/01/2022,00:09,Regular,Doméstico,Aterrizaje,AER,GAL,AEROLINEAS ARGENTINAS SA,BO-B737-81D,73,DEFINITIVO
4,01/01/2022,00:09,Regular,Internacional,Despegue,EZE,KDFW,AMERICAN AIRLINES INC.,0,261,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...
222922,30/06/2022,23:59,Regular,Doméstico,Despegue,DOZ,AER,AEROLINEAS ARGENTINAS SA,BO-B737-8SH,"83,5",PROVISORIO
222923,30/06/2022,16:26,Regular,Doméstico,Despegue,VIE,BAR,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"33,5",PROVISORIO
222924,30/06/2022,13:08,Regular,Doméstico,Aterrizaje,VIE,AER,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"45,5",PROVISORIO
222925,30/06/2022,01:33,Regular,Doméstico,Despegue,LAR,AER,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"47,5",PROVISORIO


In [41]:
# Validamos si hay valores faltantes
isna = []
for x in cols:
  isna.append( df_2022[df_2022[x].isna()] )
isna

[Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Ori

In [43]:
# Validamos si hay valores nulos
nulos = []
for x in cols:
  nulos.append( df_2022[df_2022[x].isnull()] )
nulos

[Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Origen / Destino, Aerolinea Nombre, Aeronave, Pasajeros, Calidad dato]
 Index: [],
 Empty DataFrame
 Columns: [Fecha, Hora UTC, Clase de Vuelo (todos los vuelos), Clasificación Vuelo, Tipo de Movimiento, Aeropuerto, Ori